In [1]:
from __future__ import print_function

import sys
sys.path.insert(0, 'src')

import os, random, subprocess, evaluate, shutil
import pdb

import transform, numpy as np, vgg
import scipy.misc
import tensorflow as tf
from utils import save_img, get_img, exists, list_files
from collections import defaultdict
import time
import json
import subprocess
import natsort

print("tf version:",tf.__version__)

tf version: 1.2.1


In [2]:
checkpoint_dir='./models/ckpt_cubist_b20_e4_cw05' # the saved style transformation model
path_video_in='./video_in/test_video.mp4' # the input video

# the output videl will be saved here. If this video already exists, delete it. 
path_video_out='./video_out/video.mp4'

# this folder will be used to store the frames
tmp_dir= './tmpdir_frames/'

# GPU id
device='/gpu:0'

# this many images will be processed in one batch
batch_size=4

In [3]:
# input frames will be stored here
in_dir = os.path.join(tmp_dir, 'in')

# output frames will be stored here
out_dir = os.path.join(tmp_dir, 'out')

if not os.path.exists(in_dir):
    os.makedirs(in_dir)
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
print(in_dir)
print(out_dir)

./tmpdir_frames/in
./tmpdir_frames/out


In [4]:
# let us use ffmpeg to get the frames from the video
in_args = [
    'ffmpeg',
    '-i', path_video_in,
    '%s/frame_%%d.png' % in_dir
]

In [5]:
subprocess.call(" ".join(in_args), shell=True)

0

In [6]:
base_names = natsort.natsorted(list_files(in_dir))
print(base_names[0],',...,', base_names[-1])

print(in_dir)
in_files = [os.path.join(in_dir, x) for x in base_names]
out_files = [os.path.join(out_dir, x) for x in base_names]

print(out_files[0],',...,',out_files[-1])

frame_1.png ,..., frame_884.png
./tmpdir_frames/in
./tmpdir_frames/out/frame_1.png ,..., ./tmpdir_frames/out/frame_884.png


In [7]:
#evaluate.ffwd(in_files, out_files, checkpoint_dir, device_t=device, batch_size=batch_size)

In [8]:
assert len(out_files) > 0
#is_paths = type(in_files[0]) == str

img_shape = get_img(in_files[0]).shape
print(img_shape)

(480, 854, 3)


In [10]:
# Create a tensorflow graph
g = tf.Graph()
batch_size = min(len(out_files), batch_size)
soft_config = tf.ConfigProto(allow_soft_placement=True)
soft_config.gpu_options.allow_growth = True

with g.as_default(), g.device(device), tf.Session(config=soft_config) as sess:
        
        #the current batch will have these dimensions
        batch_shape = (batch_size,) + img_shape
        print("batch_shape", batch_shape)
        
        # image input of the nural network
        img_placeholder = tf.placeholder(tf.float32, shape=batch_shape, name='img_placeholder')
        
        # the output image of the neural network
        preds = transform.net(img_placeholder)
        saver = tf.train.Saver()

        if os.path.isdir(checkpoint_dir):
            ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
            if ckpt and ckpt.model_checkpoint_path:
                # load the current model!
                saver.restore(sess, ckpt.model_checkpoint_path)
            else:
                raise Exception("No checkpoint found...")
        else:
            raise Exception("No checkpoint directory found...")
            #saver.restore(sess, checkpoint_dir)
        
        # We are going to trasnfer the style of num_iters*batch_size frames
        #num_iters = int(len(out_files)/batch_size)
        num_iters=100

        # main loop starts
        for i in range(num_iters):
            sys.stdout.write("\r transformed image: %d / %d" % (i+1, num_iters))
            sys.stdout.flush()
    
            pos = i * batch_size
            curr_batch_out = out_files[pos:pos+batch_size]
            curr_batch_in = in_files[pos:pos+batch_size]
            
            # Let us create the input of the neural network
            X = np.zeros(batch_shape, dtype=np.float32)
            for j, path_img_in in enumerate(curr_batch_in):
                # get the image from the file name
                img = get_img(path_img_in)
                assert img.shape == img_shape,'Images have different dimensions. ' +'Resize images or use --allow-different-dimensions.'
                X[j] = img
            
            # do the style transform! Put the results into preds placeholder
            _preds = sess.run(preds, feed_dict={img_placeholder:X})
            
            # save the results
            for j, path_img_out in enumerate(curr_batch_out):
                save_img(path_img_out, _preds[j])
                

batch_shape (4, 480, 854, 3)
INFO:tensorflow:Restoring parameters from ./models/ckpt_cubist_b20_e4_cw05/fns.ckpt
 transformed image: 100 / 100

In [11]:
# make a movie from the saved frames
fr = 30 
out_args = [
    'ffmpeg',
    '-i', '%s/frame_%%d.png' % out_dir,
    '-f', 'mp4',
    '-q:v', '0',
    '-vcodec', 'mpeg4',
    '-r', str(fr),
    path_video_out
]

subprocess.call(" ".join(out_args), shell=True)


1

In [12]:
print('Video at: %s' % path_video_out)

Video at: ./video_out/video.mp4


In [13]:
# delete the tmp folder
shutil.rmtree(tmp_dir)